I am continously imporving -> **This kernel is Work in Progress**

*What to expect in this Kernel:*

- I have created this kernel using **pyecharts** a new visualization library which is built on [Apache Echarts](http://echarts.apache.org/en/index.html)
- This is a very new library and i wanted to try it out for a side project because of the cool animantions it has.
- A huge shout out to the [team](https://pyecharts.org/#/en-us/team) who developed it.
- I hope this kernel will serve as a learning oppurtunity for kagglers who want to try out this new library.
- Most of the visualizations are taken inspirations from various kagglers - [@dwin183287](https://www.kaggle.com/dwin183287), [@michau96](https://www.kaggle.com/michau96), [@chandramanaha](https://www.kaggle.com/chandramanaha),[@spitfire2nd](https://www.kaggle.com/spitfire2nd)

**References are below-> Do checkout their kernels**

In [ ]:
!pip install pyecharts

In [ ]:
import csv
import json
import re
import numpy as np
import pandas as pd
from pyecharts.charts import Bar,Pie, Timeline, Scatter,Radar, HeatMap, Grid, Tab, Line, MapGlobe
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType
from pyecharts import options as opts
from collections import Counter, OrderedDict

from collections import Counter, OrderedDict
from IPython.display import HTML
pd.set_option('max_rows', 300)

# The below is great for working but if you publish it, no charts show up.
# The workaround in the next cell deals with this.
#alt.renderers.enable('notebook')

HTML("This code block contains import statements and setup.")

In [ ]:
df_2017 = pd.read_csv('../input/kaggle-survey-2017/multipleChoiceResponses.csv', encoding='latin-1')
df_2018 = pd.read_csv('../input/kaggle-survey-2018/multipleChoiceResponses.csv')
df_2019 = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv')
df_2018 = df_2018.drop([0])
df_2019 = df_2019.drop([0])

In [ ]:
df_responses = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')
df_responses = df_responses.drop([0])
df_responses.head()

In [ ]:
num_responses = [df_2017.shape[0], df_2018.shape[0], df_2019.shape[0], df_responses.shape[0]]
years = ["Year: 2017","Year: 2018","Year: 2019","Year: 2020"]

bar=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(years)\
.add_yaxis('Total Responses', num_responses)\
.set_global_opts(title_opts=opts.TitleOpts(title="Total Number of survey respondents",subtitle="2017-2020 Kaggle Survey"),
             xaxis_opts=opts.AxisOpts(name='Year'), legend_opts=opts.LegendOpts(is_show=False))

line = (Line().add_xaxis(xaxis_data=years).add_yaxis(series_name="responses",y_axis=num_responses, label_opts=opts.LabelOpts(is_show=False)))
bar.overlap(line)
bar.render_notebook()

2018 has highest number of survey participants and have `3553` respondents more than 2020

## Age Group Disribution

- Maximum number of respondents come from the 25-29 age group

In [ ]:
q1 = df_responses['Q1'].value_counts().sort_index()
q1=q1.reset_index()
q1.columns=['Age-Group', '# of responses']
q1['pct'] = round(100 * q1['# of responses']/q1['# of responses'].sum())

bar=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(q1['Age-Group'].values.tolist())\
.add_yaxis('num responses', q1['# of responses'].values.tolist())\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Age Group"),
             yaxis_opts=opts.AxisOpts(name='Age-Group'), legend_opts=opts.LegendOpts(is_show=False))\
.reversal_axis()\
.set_series_opts(
        label_opts=opts.LabelOpts(position="inside"),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="Maximum no. of responses"),
            ]
        ),
    )

pie = (
    Pie()
    .add(
        series_name="percent responses",
        data_pair=[list(z) for z in zip(q1['Age-Group'], q1.pct)],
        radius=["20%","30%"],
        center=["65%", "35%"],
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))
)

bar.overlap(pie)

bar.render_notebook()

## Education Level distribution

- approx 77% of the survey respondents are holding a degree

In [ ]:
df_responses['Q4'] = df_responses['Q4'].map({'Master’s degree': 'Master',
                                       'Bachelor’s degree': 'Bachelor', 'Doctoral degree': 'Higher than Master',
                                       'Some college/university study without earning a bachelor’s degree': 'Lower than Bachelor',
                                      'Professional degree': 'Higher than Master', 'No formal education past high school':'Lower than Bachelor'})

q4=df_responses['Q4'].value_counts().to_frame()
q4=q4.reset_index()
q4.columns=['Degree', '# of responses']
q4['pct'] = round(100 * q4['# of responses']/q4['# of responses'].sum())

bar=Bar(init_opts=opts.InitOpts(width ="600px")).add_xaxis(q4.Degree.values.tolist())\
.add_yaxis('', q4['# of responses'].values.tolist())\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Education Level",
                                           subtitle="2020 Kaggle Survey"), 
                 xaxis_opts=opts.AxisOpts(name='Degree', axislabel_opts=opts.LabelOpts(rotate=-20)),
                 legend_opts=opts.LegendOpts(is_show=False),)\

pie = (
    Pie()
    .add(
        series_name="percent responses",
        data_pair=[list(z) for z in zip(q4.Degree, q4.pct)],
        radius=["10%","20%"],
        center=["75%", "25%"],
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))
)

bar.overlap(pie)

bar.render_notebook()

## Distribution based on Sex

- majority of the survey respondents are `male`

In [ ]:
df_responses['Q2'] = df_responses['Q2'].apply(lambda x : 'ETC' if x not in ['Man', 'Woman'] else x)
total_dist=df_responses.Q2.value_counts(normalize=True)
data_q1q2 = df_responses[df_responses['Q2'] != 'ETC'].groupby(['Q2'])['Q1'].value_counts().unstack().sort_index()
man = data_q1q2.loc['Man']
woman = data_q1q2.loc['Woman']

man=man.reset_index()
man.columns = ['Age-Group', '# of responses']

woman=woman.reset_index()
woman.columns = ['Age-Group', '# of responses']

In [ ]:
total_dist = total_dist.reset_index()
total_dist.columns = ["Group", "Percent"]
total_dist.Percent = np.round((total_dist.Percent * 100),2)
total_dist['Group'] = total_dist['Group'].apply(lambda x : 'ETC' if x not in ['Man', 'Woman'] else x)
total_dist['Group'] = total_dist.Group.map({"Man":"Men","Woman":"Women", "ETC":"ETC"})

In [ ]:
bar=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(man['Age-Group'].values.tolist())\
.add_yaxis('Men', man['# of responses'].values.tolist(), stack="stack1")\
.add_yaxis('Women', woman['# of responses'].values.tolist(), stack="stack1")\
.set_series_opts(label_opts=opts.LabelOpts(position="inside"))\
.reversal_axis()\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Sex"),
                 xaxis_opts=opts.AxisOpts(name='# responses'), 
                 yaxis_opts=opts.AxisOpts(name='Age-Group'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"))\

pie = (
    Pie()
    .add(
        series_name="percent responses",
        data_pair=[list(z) for z in zip(total_dist.Group, total_dist.Percent)],
        radius=["10%","20%"],
        center=["60%", "25%"],
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))
)

bar.overlap(pie)

bar.render_notebook()

## Distribution based on Country

In [ ]:
continents_df = pd.read_csv("../input/2020-kaggle-survey-supporting/2020 Continents.csv")
continents_dict = d = {k:v for k,v in zip(continents_df["Country"],continents_df["Continents"])}


country_long_name_dict = {"United States of America": "USA", 
                          "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                          "Iran, Islamic Republic of...": "Iran"
                         }

df_2017["Continents"] = df_2017["Country"].map(continents_dict)
df_2018["Continents"] = df_2018["Q3"].map(continents_dict)
df_2019["Continents"] = df_2019["Q3"].map(continents_dict)
df_responses["Continents"] = df_responses["Q3"].map(continents_dict)

df_2017["Country"] = df_2017["Country"].replace(country_long_name_dict)
df_2018["Q3"] = df_2018["Q3"].replace(country_long_name_dict)
df_2019["Q3"] = df_2019["Q3"].replace(country_long_name_dict)
df_responses["Q3"] = df_responses["Q3"].replace(country_long_name_dict)

In [ ]:
df_2017['year'] = 2017
df_2017.rename(columns={"Country":"Q3","GenderSelect":"Q2"}, inplace=True)
df_2018['year'] = 2018
df_2018.rename(columns={"Q1":"Q2","Q2":"Q1"}, inplace=True)
df_2019['year'] = 2019
df_responses['year'] = 2020

total_responses = pd.concat([df_2017[['Q2','Q3','year']],
                             df_2018[['Q2','Q3','year']],
                             df_2019[['Q2','Q3','year']],
                             df_responses[['Q2','Q3','year']]])

total_responses=total_responses.groupby(['year','Q3']).count()

In [ ]:
total_responses = total_responses.reset_index()
total_responses.columns = ["year", "country", "count"]
total_responses.sort_values(by=["year","count"], inplace=True, ascending=[True, True])

In [ ]:
tl = Timeline()
for i in range(2017, 2021):
    data = total_responses.loc[total_responses.year == i].tail(20)
    bar=Bar(init_opts=opts.InitOpts(width ="600px")).add_xaxis(data['country'].values.tolist())\
    .add_yaxis('num responses', data['count'].values.tolist())\
    .set_series_opts(label_opts=opts.LabelOpts(position="inside"))\
    .reversal_axis()\
    .set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents wrt Country", subtitle="2017 - 2020 Kaggle Survey"),
                     xaxis_opts=opts.AxisOpts(name='Count'), legend_opts=opts.LegendOpts(is_show=False))
    grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_left="20%", pos_right="20%")))
    
    tl.add(grid, "year:{}".format(i))
    
tl.add_schema(is_auto_play=True, play_interval=2000)
tl.render_notebook()

In 2017 and 2018 - `US` is the predominant country of the survey respondents
But from 2019 onwards - `India` goes to number 1 position and followed by `US`

In [ ]:
data_2020=df_responses[['Q2','Q3','year']]
data_2020.loc[data_2020["Q3"].str.contains("China"), "Q3"]="China"
data_2020.loc[data_2020["Q3"].str.contains("USA"), "Q3"]="United States"
data_2020 = data_2020.Q3.value_counts()
high=data_2020.max()
low = data_2020.min()
data_2020= data_2020.reset_index().values.tolist()

In [ ]:
c = (
    MapGlobe()
    .add_schema()
    .add(
        maptype="world",
        series_name="World Responses",
        data_pair=data_2020,
        is_map_symbol_show=False,
        label_opts=opts.LabelOpts(is_show=False, formatter="{c}"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Globe Map - Number of respondents", subtitle="2020 Kaggle Survey"),
        visualmap_opts=opts.VisualMapOpts(
            min_=low,
            max_=high,
            range_text=["max", "min"],
            is_calculable=True,
            range_color=["lightskyblue", "yellow", "orangered"],
        )
    ))
    
c.render_notebook()

In [ ]:
#tl = Timeline()

#data_dict={2017: df_2017, 2018: df_2018, 2019:df_2019, 2020:df_responses}

#for i in range(2017, 2021):
data = df_responses

asia_df = data[data["Continents"]=="Asia"]
america_df = data[data["Continents"]=="America"]
europe_df = data[data["Continents"]=="Europe"]
africa_df = data[data["Continents"]=="Africa"]
australia_df = data[data["Continents"]=="Australia"]
others_df = data[data["Continents"]=="Others"]

continents_count_df = pd.DataFrame(data["Q2"].value_counts())
continents_count_df = continents_count_df.reset_index(drop=False)
continents_count_df.columns = ["Gender","Count"]

asia_country_count_df = pd.DataFrame(asia_df["Q2"].value_counts())
asia_country_count_df = asia_country_count_df.reset_index(drop=False)
asia_country_count_df.columns = ["Gender","Count"]

america_country_count_df = pd.DataFrame(america_df["Q2"].value_counts())
america_country_count_df = america_country_count_df.reset_index(drop=False)
america_country_count_df.columns = ["Gender","Count"]

europe_country_count_df = pd.DataFrame(europe_df["Q2"].value_counts())
europe_country_count_df = europe_country_count_df.reset_index(drop=False)
europe_country_count_df.columns = ["Gender","Count"]

others_country_count_df = pd.DataFrame(others_df["Q2"].value_counts())
others_country_count_df = others_country_count_df.reset_index(drop=False)
others_country_count_df.columns = ["Gender","Count"]

africa_country_count_df = pd.DataFrame(africa_df["Q2"].value_counts())
africa_country_count_df = africa_country_count_df.reset_index(drop=False)
africa_country_count_df.columns = ["Gender","Count"]

australia_country_count_df = pd.DataFrame(australia_df["Q2"].value_counts())
australia_country_count_df = australia_country_count_df.reset_index(drop=False)
australia_country_count_df.columns = ["Gender","Count"]

pie_world = (
    Pie()
    .add(
        "World",
        [list(z) for z in zip(continents_count_df.Gender, continents_count_df.Count)],
        center=["10%", "50%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="World", position="center"),
    )
    .add(
        "Asia",
        [list(z) for z in zip(asia_country_count_df.Gender, asia_country_count_df.Count)],
        center=["30%", "30%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Asia", position="center"),
    )
    .add(
        "Europe",
        [list(z) for z in zip(europe_country_count_df.Gender, europe_country_count_df.Count)],
        center=["50%", "30%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Europe", position="center"),
    )
    .add(
        "Africa",
        [list(z) for z in zip(africa_country_count_df.Gender, africa_country_count_df.Count)],
        center=["70%", "30%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Africa", position="center"),
    )
    .add(
        "America",
        [list(z) for z in zip(america_country_count_df.Gender, america_country_count_df.Count)],
        center=["30%", "70%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="America", position="center"),
    )
    .add(
        "Australia",
        [list(z) for z in zip(australia_country_count_df.Gender, australia_country_count_df.Count)],
        center=["50%", "70%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Australia", position="center"),
    )
    .add(
        "Others",
        [list(z) for z in zip(others_country_count_df.Gender, others_country_count_df.Count)],
        center=["70%", "70%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Others", position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Gender Distribution",subtitle="wrt Continents in 2020 Kaggle survey" ),
        legend_opts=opts.LegendOpts(
            type_="scroll", pos_top="5%", pos_left="40%", orient="horizontal"
        ),
    )
)

pie_world.render_notebook()

#tl.add(pie_world, "year:{}".format(i))
    
#tl.add_schema(is_auto_play=True, play_interval=2000)
#tl.render_notebook()

The survey is Male dominated over various continents

## Visualization Libraries

In [ ]:
viz_lst = ["Q14_Part_1", "Q14_Part_2", "Q14_Part_3", "Q14_Part_4", "Q14_Part_5", "Q14_Part_6", "Q14_Part_7", "Q14_Part_8", 
             "Q14_Part_9", "Q14_Part_10", "Q14_Part_11", "Q14_OTHER"] 
viz_df = df_responses.groupby(["Continents"])[viz_lst].count()
viz_df.columns = ["Matplotlib", "Seaborn", "Plotly/Plotly Express", "Ggplot/ggplot2", "Shiny", "D3 js", "Altair",
                  "Bokeh", "Geoplotlib", "Leaflet/Folium", "None", "Other"]
viz_df = viz_df.loc[["Asia", "America", "Europe", "Others", "Africa", "Australia"], :]
viz_df=viz_df.T

In [ ]:
hm = (
    HeatMap()
    .add_xaxis(viz_df.index.to_list())
    .add_yaxis(
        "num responses",
        ["Asia", "America", "Europe", "Others", "Africa", "Australia"],
        pd.melt(viz_df.reset_index(), id_vars='index').values.tolist(),
        label_opts=opts.LabelOpts(is_show=True, position="inside"),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)),
        title_opts=opts.TitleOpts(title="Popular Visualization Libraries", subtitle="wrt Continents" ),
        legend_opts=opts.LegendOpts(is_show=False),
        visualmap_opts=opts.VisualMapOpts(
            min_=0, max_=10000, is_calculable=True, orient="horizontal", pos_top="2%", pos_right="45%"
        )
    )
)
hm.render_notebook()

grid = (
    Grid()
    .add(hm, grid_opts=opts.GridOpts(pos_bottom="20%", pos_right="25%"))
)
grid.render_notebook()

Anywhere in the world - `Matplotlib` dominates as a popular library followed by `Seaborn` -> Hopefully `pyecharts` joins the list soon

In [ ]:
viz_lst = ["Q14_Part_1", "Q14_Part_2", "Q14_Part_3", "Q14_Part_4", "Q14_Part_5", "Q14_Part_6", "Q14_Part_7", "Q14_Part_8", 
             "Q14_Part_9", "Q14_Part_10", "Q14_Part_11", "Q14_OTHER"] 
viz_df = df_responses.groupby(["Q5"])[viz_lst].count()
viz_df.columns = ["Matplotlib", "Seaborn", "Plotly/Plotly Express", "Ggplot/ggplot2", "Shiny", "D3 js", "Altair",
                  "Bokeh", "Geoplotlib", "Leaflet/Folium", "None", "Other"]

In [ ]:
hm = (
    HeatMap()
    .add_xaxis(viz_df.index.to_list())
    .add_yaxis(
        "num responses",
        viz_df.columns.tolist(),
        pd.melt(viz_df.reset_index(), id_vars='Q5').values.tolist(),
        label_opts=opts.LabelOpts(is_show=True, position="inside"),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)),
        title_opts=opts.TitleOpts(title="Popular Visualization Libraries", subtitle="wrt Profession" ),
        legend_opts=opts.LegendOpts(is_show=False),
        visualmap_opts=opts.VisualMapOpts(
            min_=0, max_=4000, is_calculable=True, orient="horizontal", pos_top="2%", pos_right="45%"
        )
    )
)
#hm.render_notebook()

grid = (
    Grid()
    .add(hm, grid_opts=opts.GridOpts(pos_bottom="20%", pos_left="15%", pos_right="20%"))
)
grid.render_notebook()

Almost all the professionals and students use `Matplotlib` as the commmon visualization tool as expected

## Distribution based on Profession

In [ ]:
q5=df_responses['Q5'].value_counts().to_frame()
q5=q5.reset_index()
q5.columns=['Profession', '# of responses']
q5['pct'] = round(100 * q5['# of responses']/q5['# of responses'].sum())

bar=Bar(init_opts=opts.InitOpts(width ="600px")).add_xaxis(q5.Profession.values.tolist())\
.add_yaxis('', q5['# of responses'].values.tolist())\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Profession",
                                           subtitle="2020 Kaggle Survey"), 
                 xaxis_opts=opts.AxisOpts(name='Degree', axislabel_opts=opts.LabelOpts(rotate=-25)),
                 legend_opts=opts.LegendOpts(is_show=False),)\
.set_series_opts(label_opts=opts.LabelOpts(position="inside"))

pie = (
    Pie()
    .add(
        series_name="percent responses",
        data_pair=[list(z) for z in zip(q5.Profession, q5.pct)],
        radius=["10%","20%"],
        center=["65%", "25%"],
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))
)

bar.overlap(pie)

grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="25%", pos_right="35%"))
)
grid.render_notebook()

## Programming Experience

In [ ]:
q6=df_responses['Q6'].value_counts().to_frame()
q6=q6.reset_index()
q6.columns=['Coding Experience', '# of responses']
q6['pct'] = round(100 * q6['# of responses']/q6['# of responses'].sum())

bar=Bar(init_opts=opts.InitOpts(width ="600px")).add_xaxis(q6["Coding Experience"].values.tolist())\
.add_yaxis('', q6['# of responses'].values.tolist())\
.set_global_opts(title_opts=opts.TitleOpts(title="Programming Experience of the respondents",
                                           subtitle="2020 Kaggle Survey"), 
                 xaxis_opts=opts.AxisOpts(name='experience', axislabel_opts=opts.LabelOpts(rotate=-25)),
                 legend_opts=opts.LegendOpts(is_show=False),)\
.set_series_opts(label_opts=opts.LabelOpts(position="inside"))

pie = (
    Pie()
    .add(
        series_name="percent responses",
        data_pair=[list(z) for z in zip(q6["Coding Experience"], q6.pct)],
        radius=["10%","20%"],
        center=["55%", "25%"],
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))
)

bar.overlap(pie)

grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="25%", pos_right="35%"))
)
grid.render_notebook()

Most of the survey participants have atleast `3-5 years` of programming experience

## Professionals v Students

In [ ]:
dataprofs = df_responses[df_responses["Q5"]!="Student"]
students = df_responses[df_responses["Q5"]=="Student"]

q7 = [col for col in df_responses.columns if 'Q7' in col]


languages = []
for qn in q7:
    for val in dataprofs[qn].unique():
        languages.append(val)
        
languages = [lang for lang in languages if str(lang)!='nan']

prof_langs = round(100 * ((dataprofs.shape[0] - dataprofs[q7].isnull().sum()) / dataprofs.shape[0]), 2)
student_langs = round(100 *((students.shape[0] - students[q7].isnull().sum()) / students.shape[0]), 2)

prof_langs = prof_langs.reset_index()
prof_langs['language'] = languages
prof_langs.rename(columns={0: "pct_prof"}, inplace=True)
prof_langs.sort_values(by=["pct_prof"], inplace=True)
student_langs = student_langs.reset_index()
student_langs['language'] = languages
student_langs.rename(columns={0: "pct_stud"}, inplace=True)
student_langs.sort_values(by=["pct_stud"], inplace=True)

langs=pd.merge(prof_langs, student_langs, on='language')

In [ ]:
profs_recom_lang = dataprofs['Q8'].value_counts().reset_index()
profs_recom_lang['prof_pct'] = round(100*(profs_recom_lang['Q8']/profs_recom_lang['Q8'].sum()),2)
studs_recom_lang = students['Q8'].value_counts().reset_index()
studs_recom_lang['stud_pct'] = round(100*(studs_recom_lang['Q8']/studs_recom_lang['Q8'].sum()),2)
rcom_lang=pd.merge(profs_recom_lang, studs_recom_lang, on='index')
rcom_lang.sort_values(by=["prof_pct"], inplace=True)

In [ ]:
bar=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(langs['language'].values.tolist())\
.add_yaxis('Data Professionals', langs['pct_prof'].values.tolist())\
.add_yaxis('Students', langs['pct_stud'].values.tolist())\
.set_series_opts(label_opts=opts.LabelOpts(position="right", formatter="{c}%"))\
.reversal_axis()\
.set_global_opts(title_opts=opts.TitleOpts(title="Popular Programming Language"),
                 xaxis_opts=opts.AxisOpts(name='% pct'), 
                 yaxis_opts=opts.AxisOpts(name='Language'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="50%", pos_right="30%", orient="vertical"))

bar2=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(rcom_lang['index'].values.tolist())\
.add_yaxis('Data Professionals', rcom_lang['prof_pct'].values.tolist())\
.add_yaxis('Students', rcom_lang['stud_pct'].values.tolist())\
.set_series_opts(label_opts=opts.LabelOpts(position="right", formatter="{c}%"))\
.reversal_axis()\
.set_global_opts(title_opts=opts.TitleOpts(title="Popular Recommended Languages"),
                 xaxis_opts=opts.AxisOpts(name='% pct'), 
                 yaxis_opts=opts.AxisOpts(name='Language'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="50%", pos_right="50%", orient="vertical"))

grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_left="10%", pos_right="15%"))
)

grid2 = (
    Grid()
    .add(bar2, grid_opts=opts.GridOpts(pos_left="10%", pos_right="15%"))
)

tab = Tab()

tab.add(grid, "Current Language")
tab.add(grid2, "Recommended Language")

In [ ]:
q9 = [col for col in df_responses.columns if 'Q9' in col]

ides = []
for qn in q9:
    for val in dataprofs[qn].unique():
        ides.append(val)
        
ides = [ide.strip() for ide in ides if str(ide)!='nan']

prof_ides = round(100 * ((dataprofs.shape[0] - dataprofs[q9].isnull().sum()) / dataprofs.shape[0]), 2)
student_ides = round(100 *((students.shape[0] - students[q9].isnull().sum()) / students.shape[0]), 2)

prof_ides = prof_ides.reset_index()
prof_ides['ides'] = ides
prof_ides.rename(columns={0: "pct_prof"}, inplace=True)
prof_ides.sort_values(by=["pct_prof"], inplace=True)
student_ides = student_ides.reset_index()
student_ides['ides'] = ides
student_ides.rename(columns={0: "pct_stud"}, inplace=True)
student_ides.sort_values(by=["pct_stud"], inplace=True)

ides_df = pd.merge(prof_ides, student_ides, on='ides')
ides_df.loc[ides_df["ides"].str.contains('Jupyter'), "ides"] = "Jupyter"
ides_df.loc[ides_df["ides"].str.contains('(VSCode)'), "ides"] = "VSCode"

In [ ]:
q10 = [col for col in df_responses.columns if 'Q10' in col]

nbs = []
for qn in q10:
    for val in dataprofs[qn].unique():
        nbs.append(val)
        
nbs = [nb.strip() for nb in nbs if str(nb)!='nan']

prof_nbs = round(100 * ((dataprofs.shape[0] - dataprofs[q10].isnull().sum()) / dataprofs.shape[0]), 2)
student_nbs = round(100 *((students.shape[0] - students[q10].isnull().sum()) / students.shape[0]), 2)

prof_nbs = prof_nbs.reset_index()
prof_nbs['nbs'] = nbs
prof_nbs.rename(columns={0: "pct_prof"}, inplace=True)
prof_nbs.sort_values(by=["pct_prof"], inplace=True)
student_nbs = student_nbs.reset_index()
student_nbs['nbs'] = nbs
student_nbs.rename(columns={0: "pct_stud"}, inplace=True)
student_nbs.sort_values(by=["pct_stud"], inplace=True)

nbs_df = pd.merge(prof_nbs, student_nbs, on='nbs')
nbs_df['nbs'] = nbs_df['nbs'].str.replace(" Notebooks", "").str.replace(" Studio", "")

In [ ]:
profs_envs = dataprofs['Q11'].value_counts()
profs_envs=profs_envs.reset_index()
profs_envs.rename(columns={"Q11": "pct_prof"}, inplace=True)
profs_envs["pct_prof"] = round(100*(profs_envs["pct_prof"]/profs_envs["pct_prof"].sum()),2)
profs_envs.sort_values(by=["pct_prof"], inplace=True)

student_envs = students['Q11'].value_counts()
student_envs=student_envs.reset_index()

student_envs.rename(columns={"Q11": "pct_stud"}, inplace=True)
student_envs["pct_stud"] = round(100*(student_envs["pct_stud"]/student_envs["pct_stud"].sum()),2)
student_envs.sort_values(by=["pct_stud"], inplace=True)

envs_df =pd.merge(profs_envs, student_envs, on='index')
envs_df.rename(columns={"index": "envs"}, inplace=True)

In [ ]:
q12 = [col for col in df_responses.columns if 'Q12' in col]

nbs = []
for qn in q12:
    for val in dataprofs[qn].unique():
        nbs.append(val)
        
nbs = [nb.strip() for nb in nbs if str(nb)!='nan']

prof_nbs = round(100 * ((dataprofs.shape[0] - dataprofs[q12].isnull().sum()) / dataprofs.shape[0]), 2)
student_nbs = round(100 *((students.shape[0] - students[q12].isnull().sum()) / students.shape[0]), 2)

prof_nbs = prof_nbs.reset_index()
prof_nbs['nbs'] = nbs
prof_nbs.rename(columns={0: "pct_prof"}, inplace=True)
prof_nbs.sort_values(by=["pct_prof"], inplace=True)
student_nbs = student_nbs.reset_index()
student_nbs['nbs'] = nbs
student_nbs.rename(columns={0: "pct_stud"}, inplace=True)
student_nbs.sort_values(by=["pct_stud"], inplace=True)

hardware_df = pd.merge(prof_nbs, student_nbs, on='nbs')

hardware_df.rename(columns={"nbs": "hardware"}, inplace=True)

In [ ]:
bar3=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(ides_df['ides'].values.tolist())\
.add_yaxis('Data Professionals', ides_df['pct_prof'].values.tolist())\
.add_yaxis('Students', ides_df['pct_stud'].values.tolist())\
.reversal_axis()\
.set_series_opts(label_opts=opts.LabelOpts(position="right", formatter="{c}%"))\
.set_global_opts(title_opts=opts.TitleOpts(title="Popular IDEs"),
                 yaxis_opts=opts.AxisOpts(name='IDEs', axislabel_opts=opts.LabelOpts(rotate=20)), 
                 xaxis_opts=opts.AxisOpts(name='% pct'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="50%", pos_left="50%", orient="vertical"))


bar4=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(nbs_df['nbs'].values.tolist())\
.add_yaxis('Data Professionals', nbs_df['pct_prof'].values.tolist())\
.add_yaxis('Students', nbs_df['pct_stud'].values.tolist())\
.reversal_axis()\
.set_series_opts(label_opts=opts.LabelOpts(position="right", formatter="{c}%"))\
.set_global_opts(title_opts=opts.TitleOpts(title="Popular Notebook Environments"),
                 yaxis_opts=opts.AxisOpts(name='NBs', axislabel_opts=opts.LabelOpts(rotate=20)), 
                 xaxis_opts=opts.AxisOpts(name='% pct'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="50%", pos_left="50%", orient="vertical"))

bar5=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(envs_df['envs'].values.tolist())\
.add_yaxis('Data Professionals', envs_df['pct_prof'].values.tolist())\
.add_yaxis('Students', envs_df['pct_stud'].values.tolist())\
.reversal_axis()\
.set_series_opts(label_opts=opts.LabelOpts(position="right", formatter="{c}%"))\
.set_global_opts(title_opts=opts.TitleOpts(title="Computing Platform"),
                 yaxis_opts=opts.AxisOpts(name='ENVs', axislabel_opts=opts.LabelOpts(rotate=20)), 
                 xaxis_opts=opts.AxisOpts(name='% pct'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="50%", pos_left="50%", orient="vertical"))

bar6=Bar(init_opts=opts.InitOpts(width="650px")).add_xaxis(hardware_df['hardware'].values.tolist())\
.add_yaxis('Data Professionals', hardware_df['pct_prof'].values.tolist())\
.add_yaxis('Students', hardware_df['pct_stud'].values.tolist())\
.reversal_axis()\
.set_series_opts(label_opts=opts.LabelOpts(position="right", formatter="{c}%"))\
.set_global_opts(title_opts=opts.TitleOpts(title="Popular Hardware"),
                 yaxis_opts=opts.AxisOpts(name='HWs', axislabel_opts=opts.LabelOpts(rotate=20)), 
                 xaxis_opts=opts.AxisOpts(name='% pct'),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="50%", pos_left="50%", orient="vertical"))

grid3 = (
    Grid()
    .add(bar3, grid_opts=opts.GridOpts(pos_left="10%", pos_right="25%"))
)

grid4 = (
    Grid()
    .add(bar4, grid_opts=opts.GridOpts(pos_left="20%", pos_right="15%"))
)

grid5 = (
    Grid()
    .add(bar5, grid_opts=opts.GridOpts(pos_left="30%", pos_right="15%"))
)
grid6 = (
    Grid()
    .add(bar6, grid_opts=opts.GridOpts(pos_left="10%", pos_right="15%"))
)


tab.add(grid3, "IDEs")
tab.add(grid4, "Hosted NBs")
tab.add(grid5, "Computing Platform")
tab.add(grid6, "Specialized HW")
tab.render_notebook()

**Current Programming Language**
- `Python`, `SQL`, `R` are the primary languages used by majority of the data professionals
- `Python`, `C++`, `C` are used more often by students -> could be from their course curriculum

**Recommended Programming Language**
- `Python` without any doubt

**IDEs**
- It is very clear that `Jupyter` is favoured among both the students and professionals
- `VSCode`,`Pycharm` are follwed after `Jupyter`

**Hosted Notebooks**
- `Google Colab` and `Kaggle` are most widely used


In [ ]:
data = df_responses['Q22'].value_counts()
data = data.reset_index()
data.rename(columns={"index":"response"}, inplace=True)
data['pct'] = round(100*(data['Q22']/data['Q22'].sum()),2)
data.sort_values(by='pct', inplace=True, ascending=False)
pie = (
    Pie()
    .add(
        series_name="ML Adoptions",
        data_pair=[list(z) for z in zip(data['response'], data['pct'])],
        radius=["20%","40%"],
        center=["50%", "60%"],rosetype="area"
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))\
    .set_global_opts(title_opts=opts.TitleOpts(title="ML Adoptions"),
                 legend_opts=opts.LegendOpts(type_="scroll", pos_top="3%", orient="vertical"))
)
pie.render_notebook()

### References

* [Enthusiast to Data Professional - What changes?](https://www.kaggle.com/spitfire2nd/enthusiast-to-data-professional-what-changes)
* [Kagglers seen by continents](https://www.kaggle.com/dwin183287/kagglers-seen-by-continents)
* [Education level affects data analysis? ](https://www.kaggle.com/michau96/education-level-affects-data-analysis)
* [KYC-Know YOur Community!](https://www.kaggle.com/chandramanaha/kyc-know-your-community)